# Directional Bias Amplification (A -> T)

Notebook with 2 simple examples to help understand how this bias amplification metric works.  
Metric presented in this paper: https://arxiv.org/pdf/2102.12594.pdf  
Original code repository: https://github.com/princetonvisualai/directional-bias-amp

In [1]:
# Following code is taken from https://github.com/princetonvisualai/directional-bias-amp/blob/main/directional_biasamp.py

import os
import json
import numpy as np
from typing import List 

def biasamp_attribute_to_task(task_labels: np.ndarray,
                              attribute_labels: np.ndarray,
                              task_preds: np.ndarray,
                              task_labels_train=None,
                              attribute_labels_train=None,
                              names: List[List]=None):
    '''
    for each of the following, an entry of 1 is a prediction, and 0 is not
    task_labels: n x |T|, these are labels on the test set, where n is the number of samples, and |T| is the number of tasks to be classified
    attribute_labels: n x |A|, these are labels on the test set, where n is the number of samples, and |A| is the number of attributes to be classified
    task_preds: n x |T|, these are predictions on the test set for task

    optional: below are used for setting the direction of the indicator variable. if not provided, test labels are used
    task_labels_train: m x |T|, these are labels on the train set, where m is the number of samples, and |T| is the number of tasks to be classified
    attribute_labels_train: m x |A|, these are labels on the train set, where m is the number of samples, and |A| is the number of attributes to be classified

    names: list of [task_names, attribute_names]. if included, will print out the top 10 attribute-task pairs with the most bias amplification
    '''

    assert len(task_labels.shape) == 2 and len(
        attribute_labels.shape) == 2, 'Please read the shape of the expected inputs, which should be "num samples" by "num classification items"'
    if task_labels_train is None or attribute_labels_train is None:
        task_labels_train, attribute_labels_train = task_labels, attribute_labels
    num_t, num_a = task_labels.shape[1], attribute_labels.shape[1]

    # only include images that have attribute(s) and task(s) associated with it for calculation of indicator variable
    keep_indices = np.array(list(set(np.where(np.sum(task_labels_train, axis=1) > 0)[0]).union(
        set(np.where(np.sum(attribute_labels_train, axis=1) > 0)[0]))))
    task_labels_train, attribute_labels_train = task_labels_train[keep_indices], attribute_labels_train[keep_indices]

    # y_at calculation
    p_at = np.zeros((num_a, num_t))
    p_a_p_t = np.zeros((num_a, num_t))
    num_train = len(task_labels_train)
    for a in range(num_a):
        for t in range(num_t):
            t_indices = np.where(task_labels_train[:, t] == 1)[0]
            a_indices = np.where(attribute_labels_train[:, a] == 1)[0]
            at_indices = set(t_indices) & set(a_indices)
            p_a_p_t[a][t] = (len(t_indices) / num_train) * (len(a_indices) / num_train)
            p_at[a][t] = len(at_indices) / num_train
    print("p_at\n", p_at)
    print("p_a_p_t\n", p_a_p_t)
    y_at = np.sign(p_at - p_a_p_t)
    print("y_at\n", y_at)

    # delta_at calculation
    t_cond_a = np.zeros((num_a, num_t))
    that_cond_a = np.zeros((num_a, num_t))
    for a in range(num_a):
        for t in range(num_t):
            t_cond_a[a][t] = np.mean(task_labels[:, t][np.where(attribute_labels[:, a] == 1)[0]])
            that_cond_a[a][t] = np.mean(task_preds[:, t][np.where(attribute_labels[:, a] == 1)[0]])
    delta_at = that_cond_a - t_cond_a
    print("delta_at\n", delta_at)
    
    values = y_at * delta_at
    print("values\n", values)
    print()
    val = np.nanmean(values)

    if names is not None:
        assert len(names) == 2, "Names should be a list of the task names and attribute names"
        task_names, attribute_names = names
        assert len(task_names) == num_t and len(
            attribute_names) == num_a, "The number of names should match both the number of tasks and number of attributes"

        sorted_indices = np.argsort(np.absolute(values).flatten())
        for i in sorted_indices[::-1][:10]:
            a, t = i // num_t, i % num_t
            print("{0} - {1}: {2:.4f}".format(attribute_names[a], task_names[t], values[a][t]))
    return val



## Example 1)

In [2]:
# Initialize variables
n = 100+10 
mapping_obj = {"kitchen": 0, "pc":1, "pizza":2, "surf":3}
task_labels = np.zeros((n, len(mapping_obj)))
attribute_labels = np.zeros((n, 2))
task_preds = np.zeros((n, len(mapping_obj)))
attribute_preds = np.zeros((n, 2))

man_object = {"kitchen": 5, "pc":1, "pizza":80, "surf":14}
woman_object = {"kitchen": 2, "pc":1, "pizza":0, "surf":7}

man_object_pred = {"kitchen": 5, "pc":2, "pizza":65, "surf":28}
woman_object_pred = {"kitchen": 5, "pc":0, "pizza":0, "surf":5}

# Fill-in values
task = 0
for obj, counts in man_object.items():
    for i in range(counts):
        task_labels[task][mapping_obj[obj]] = 1
        task += 1
for obj, counts in woman_object.items():
    for i in range(counts):
        task_labels[task][mapping_obj[obj]] = 1
        task += 1
task = 0
for obj, counts in man_object_pred.items():
    for i in range(counts):
        task_preds[task][mapping_obj[obj]] = 1
        task += 1
for obj, counts in woman_object_pred.items():
    for i in range(counts):
        task_preds[task][mapping_obj[obj]] = 1
        task += 1
        
for q_idx in range(sum(man_object_pred.values())):
    attribute_labels[q_idx][0] = 1

for q_idx in range(sum(woman_object_pred.values())):
    attribute_labels[q_idx+sum(man_object_pred.values())][1] = 1


task_labels_train=None
attribute_labels_train=None
names=[list(mapping_obj.keys()), ["Male", "Female"]]

dba_gender_to_obj = biasamp_attribute_to_task(task_labels, 
                                                 attribute_labels, 
                                                 task_preds, 
                                                 task_labels_train,
                                                 attribute_labels_train,
                                                 names)
print("dba_gender_to_object", dba_gender_to_obj)

p_at
 [[0.04545455 0.00909091 0.72727273 0.12727273]
 [0.01818182 0.00909091 0.         0.06363636]]
p_a_p_t
 [[0.05785124 0.01652893 0.66115702 0.17355372]
 [0.00578512 0.00165289 0.0661157  0.01735537]]
y_at
 [[-1. -1.  1. -1.]
 [ 1.  1. -1.  1.]]
delta_at
 [[ 0.    0.01 -0.15  0.14]
 [ 0.3  -0.1   0.   -0.2 ]]
values
 [[-0.   -0.01 -0.15 -0.14]
 [ 0.3  -0.1  -0.   -0.2 ]]

Female - kitchen: 0.3000
Female - surf: -0.2000
Male - pizza: -0.1500
Male - surf: -0.1400
Female - pc: -0.1000
Male - pc: -0.0100
Female - pizza: -0.0000
Male - kitchen: -0.0000
dba_gender_to_object -0.037500000000000006


## Example 2)

In [11]:
# Initialize variables
n = 100+10 # 100 men, 10 women
# n=10+14
mapping_obj = {"cat": 0, "dog":1, "horse":2} # 3 tasks
task_labels = np.zeros((n, len(mapping_obj)))
attribute_labels = np.zeros((n, 2))
task_preds = np.zeros((n, len(mapping_obj)))
# attribute_preds = np.zeros((n, 2))  # Not needed now. Evaluating A->T
# True distribution
man_object = {"cat": 10, "dog":30, "horse":60}
woman_object = {"cat": 1, "dog":2, "horse":7}
# Predictions
man_object_pred = {"cat": 28, "dog":2, "horse":70}
woman_object_pred = {"cat": 2, "dog":5, "horse":3}
# # True distribution
# man_object = {"cat": 5, "dog":5}
# woman_object = {"cat": 7, "dog":7}
# # Predictions
# man_object_pred = {"cat": 1, "dog":9}
# woman_object_pred = {"cat": 10, "dog":4}

# Fill-in values
task = 0
for obj, counts in man_object.items():
    for i in range(counts):
        task_labels[task][mapping_obj[obj]] = 1
        task += 1
for obj, counts in woman_object.items():
    for i in range(counts):
        task_labels[task][mapping_obj[obj]] = 1
        task += 1
task = 0
for obj, counts in man_object_pred.items():
    for i in range(counts):
        task_preds[task][mapping_obj[obj]] = 1
        task += 1
for obj, counts in woman_object_pred.items():
    for i in range(counts):
        task_preds[task][mapping_obj[obj]] = 1
        task += 1
        
for q_idx in range(sum(man_object_pred.values())):
    attribute_labels[q_idx][0] = 1

for q_idx in range(sum(woman_object_pred.values())):
    attribute_labels[q_idx+sum(man_object_pred.values())][1] = 1


task_labels_train=None
attribute_labels_train=None
names=[list(mapping_obj.keys()), ["Male", "Female"]]

dba_gender_to_obj = biasamp_attribute_to_task(task_labels,
                                              attribute_labels, 
                                              task_preds, 
                                              task_labels_train,
                                              attribute_labels_train,
                                              names)
print("dba_gender_to_object", dba_gender_to_obj)

p_at
 [[0.09090909 0.27272727 0.54545455]
 [0.00909091 0.01818182 0.06363636]]
p_a_p_t
 [[0.09090909 0.26446281 0.55371901]
 [0.00909091 0.02644628 0.0553719 ]]
y_at
 [[ 0.  1. -1.]
 [-1. -1.  1.]]
delta_at
 [[ 0.18 -0.28  0.1 ]
 [ 0.1   0.3  -0.4 ]]
values
 [[ 0.   -0.28 -0.1 ]
 [-0.1  -0.3  -0.4 ]]

Female - horse: -0.4000
Female - dog: -0.3000
Male - dog: -0.2800
Female - cat: -0.1000
Male - horse: -0.1000
Male - cat: 0.0000
dba_gender_to_object -0.19666666666666666
